<a href="https://colab.research.google.com/github/dgo95/TFG_IDENTIFICACION_AVES/blob/main/2_Preprocesamiento_Dataset.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install scikit-maad
!pip install pydub

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 142 kB 14.1 MB/s 
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import numpy as np
import os
import cv2  
from pydub import AudioSegment
import contextlib
import wave
import sys
import imageio
import random
import shutil
import librosa
import librosa.display
import matplotlib.pyplot as plt
import maad
from maad import sound, util, rois, features
from maad.util import (power2dB, plot2d, format_features, read_audacity_annot,
                       overlay_rois, overlay_centroid, crop_image,save_figlist)

In [ ]:
def filtroFrecuencias(powerSpectrum, frequenciesFound, fMax,fMin):
    i = 0
    min = len(powerSpectrum)-1
    for f in frequenciesFound:
        if f > fMin and i<min:
            min = i
        elif f > fMax:
            break
        i = i + 1
    l = len(powerSpectrum)
    aux = np.delete(powerSpectrum, range(i,l),0)
    return np.delete(aux, range(0,min),0) , frequenciesFound[min:i]
def iguala(img):
  anch = img.shape[1] # keep original height
  alt = img.shape[0]
  dim = (anch,ALTO)
  # resize image
  if alt < ALTO:
    img = cv2.resize(img, dim, interpolation = cv2.INTER_LINEAR)
  else:
    img = cv2.resize(img, dim, interpolation = cv2.INTER_AREA)

  dim= (ANCHO,ALTO)
  if (anch > ANCHO):
    agranda = False
  else:
    agranda = True
  if agranda:
    img = cv2.resize(img, dim, interpolation = cv2.INTER_LINEAR)
  else:
    img = cv2.resize(img, dim, interpolation = cv2.INTER_AREA)
  return img
def todoAEntrenamiento():
  for nombre_directorio, dirs, ficheros in os.walk(test):
    for nombre_fichero in ficheros:
      os.replace(os.path.join(nombre_directorio,nombre_fichero),os.path.join(nombre_directorio.replace("test","train"),nombre_fichero))
def ordena(e, t):
  todoAEntrenamiento()
  for nombre_directorio, dirs, ficheros in os.walk(e):
    idx = 0
    test = nombre_directorio.replace("train","test")
    existe = os.path.exists(test)#Se optiene si existe o no
    if not existe:  
        os.makedirs(test)
    for nombre_fichero in ficheros:
      if idx > len(ficheros) * .8:
        os.replace(os.path.join(nombre_directorio,nombre_fichero),os.path.join(nombre_directorio.replace("train","test"),nombre_fichero))
      idx = idx +1
def agrupaRois(df):
  df.sort_values(by = "min_x", ascending=True, inplace=True)
  i =-11
  borrar = []
  contador = 0
  for index, row in df.iterrows():
        if i == -11:
          i=index
          continue
        if row["min_x"]<=df._get_value(i, 'max_x'):
          df.at[i, 'max_x'] = row["max_x"] if row["max_x"] > df.at[index, 'max_x'] else df.at[i, 'max_x']
          df.at[i, 'max_t'] = row["max_t"] if row["max_t"] > df.at[index, 'max_t'] else df.at[i, 'max_t']
          df.at[i, 'min_t'] = row["min_t"] if row["min_t"] > df.at[index, 'min_t'] else df.at[i, 'min_t']
          borrar.append(index)
          contador = contador +1
        else:
          i = index
  for b in borrar:
    df = df.drop(b)
  return df
def ficherosTotales():
    contador=0
    for nombre_directorio, dirs, ficheros in os.walk(original):
        for nombre_fichero in ficheros:
            contador = contador +1
    return contador
def normalizaMaximo5DB(sound):
    change_in_dBFS = 5 - sound.max_dBFS
    nSong = sound.apply_gain(change_in_dBFS)
    return nSong

def audioMasLargo(ruta,duracion):
    audio_in_file = ruta
    audio_out_file = ruta.replace(".wav","").replace("Original","MismaDuracion")
    #lee el fichero wav
    song = AudioSegment.from_wav(audio_in_file)
    
    repeticiones= duracion // DURACION_MAXIMA
    if(repeticiones == 1):
            save_name = audio_in_file.replace("Original","MismaDuracion")
            songP = song[:DURACION_MAXIMA*1000]
            #Establece el pico máximo en 5 dB
            songP = normalizaMaximo5DB(songP)
            songP.export(save_name, format="wav")
    else:
        for i in range(1,int(repeticiones)+1):
            songP = song[DURACION_MAXIMA*1000*(i-1):DURACION_MAXIMA*1000*i]
            save_name = audio_out_file + "part"+str(i)+".wav"
            #Establece el pico máximo en 5 dB
            songP = normalizaMaximo5DB(songP)
            songP = songP.set_channels(1)
            songP.export(save_name, format="wav")

def audioMasCorto(ruta,duracion):
    audio_in_file = ruta
    
    # crear segundos de silencio necesarios
    silence_segment = AudioSegment.silent(duration=DURACION_MAXIMA*1000-round(duracion*1000))  #duracion en milisegundos

    #lee el fichero wav
    song = AudioSegment.from_wav(audio_in_file)

    #Junta los dos segmentos    
    final_song = song + silence_segment
    
    #Establece el pico máximo en 5 dB
    final_song = normalizaMaximo5DB(final_song)

    #Guarda la modificación
    final_song.export(audio_in_file.replace("Original","MismaDuracion"), format="wav")

In [ ]:
def IgualarDuraciones(ruta, maximo):
  filename, file_extension = os.path.splitext(ruta)
  if file_extension == '.csv':
    return
  try:
    with contextlib.closing(wave.open(ruta,'r')) as f:
      frames = f.getnframes()
      rate = f.getframerate()
      duration = frames / float(rate)
      if(duration<maximo):
        audioMasCorto(ruta,duration)
      else:
        audioMasLargo(ruta,duration)  
  except:
    print("Excepción en el fichero "+filename+" de la ruta "+ruta)

In [ ]:
def Espectrograma1():
  for nombre_directorio, dirs, ficheros in os.walk(original.replace("Original","MismaDuracion")):
    espectograma = nombre_directorio.replace("MismaDuracion", "Espectrogramas")
    existe = os.path.exists(espectograma)  # Se optiene si existe o no
    if not existe:
        os.makedirs(espectograma)
    for nombre_fichero in ficheros:
        ruta = os.path.join(nombre_directorio, nombre_fichero)
        try:
          s, fs = sound.load(ruta)
          dB_max = 96

          Sxx_power, tn, fn, ext = sound.spectrogram(s, fs, nperseg=1024, noverlap=1024//2)
          ext[2] = frecuenciaMinima
          ext[3] = frecuenciaMaxima
          Sxx_power , fn = filtroFrecuencias(Sxx_power, fn, frecuenciaMaxima,frecuenciaMinima)
          Sxx_power_noNoise= sound.median_equalizer(Sxx_power, display=False, **{'extent':ext})
          Sxx_db_noNoise = power2dB(Sxx_power_noNoise)
          Sxx_db_noNoise_smooth = sound.smooth(Sxx_db_noNoise, std=0.5,
                                      display=False, savefig=None,
                                      **{'vmin':0, 'vmax':dB_max, 'extent':ext})

          ruta1 = ruta.replace("MismaDuracion", "Espectrogramas").replace(".wav", ".png")
          imageio.imwrite(ruta1, (Sxx_db_noNoise_smooth).astype(np.uint8))
          img = iguala(cv2.imread(ruta1, cv2.IMREAD_GRAYSCALE))
          cv2.imwrite(ruta1, img)
          os.remove(ruta)
        except:
          print("Excepcion en ",ruta)
          continue

In [ ]:
def Espectrograma2():
  for nombre_directorio, dirs, ficheros in os.walk(original.replace("Original","MismaDuracion")):
    espectograma = nombre_directorio.replace("MismaDuracion", "Espectrogramas")
    existe = os.path.exists(espectograma)  # Se optiene si existe o no
    if not existe:
        os.makedirs(espectograma)
    for nombre_fichero in ficheros:
        ruta = os.path.join(nombre_directorio, nombre_fichero)
        path, fichero = os.path.split(ruta)
        try:
          s, fs = sound.load(ruta)
          dB_max = 96

          Sxx_power, tn, fn, ext = sound.spectrogram(s, fs, nperseg=1024, noverlap=1024//2)
          ext[2] = frecuenciaMinima
          ext[3] = frecuenciaMaxima
          Sxx_power , fn = filtroFrecuencias(Sxx_power, fn, frecuenciaMaxima,frecuenciaMinima)
          _, _, snr, _, _, _ = sound.spectral_snr(Sxx_power)
          if snr > 30 and snr < 100:
            Sxx_power_noNoise= sound.median_equalizer(Sxx_power, display=False, **{'extent':ext})
            Sxx_db_noNoise = power2dB(Sxx_power_noNoise)
            Sxx_db_noNoise_smooth = sound.smooth(Sxx_db_noNoise, std=0.5,
                                        display=False, savefig=None,
                                        **{'vmin':0, 'vmax':dB_max, 'extent':ext})

            #ruta1 = ruta.replace("MismaDuracion", "Espectrogramas").replace(".wav", ".png")
            ruta1 = os.path.join(path.replace("MismaDuracion", "Espectrogramas"),str(snr) +".png")
            imageio.imwrite(ruta1, (Sxx_db_noNoise_smooth).astype(np.uint8))
            img = iguala(cv2.imread(ruta1, cv2.IMREAD_GRAYSCALE))
            cv2.imwrite(ruta1, img)
          os.remove(ruta)
        except:
          print("Excepcion en ",ruta)
          continue

In [ ]:
def EspectrogramaMel():
  for nombre_directorio, dirs, ficheros in os.walk(original.replace("Original","MismaDuracion")):
    espectograma = nombre_directorio.replace("MismaDuracion", "Espectrogramas")
    existe = os.path.exists(espectograma)  # Se optiene si existe o no
    if not existe:
        os.makedirs(espectograma)
    for nombre_fichero in ficheros:
        ruta = os.path.join(nombre_directorio, nombre_fichero)
        path, fichero = os.path.split(ruta)
        try:
          s, fs = sound.load(ruta)
          dB_max = 96

          Sxx_power, tn, fn, ext = sound.spectrogram(s, fs, nperseg=1024, noverlap=1024//2)
          _, _, snr, _, _, _ = sound.spectral_snr(Sxx_power)
          if snr > 30 and snr < 100:
            ruta1 = os.path.join(path.replace("MismaDuracion", "Espectrogramas"),str(snr) +".png")
            signal, sr = librosa.load(ruta)
            saveMel(signal, ruta1,sr)
            img = cv2.imread(ruta1, cv2.IMREAD_UNCHANGED)
            scale_percent = 50 # percent of original size
            width = int(img.shape[1] * scale_percent / 100)
            height = int(img.shape[0] * scale_percent / 100)
            dim = (width, height)
              
            # resize image
            resized = cv2.resize(img, dim, interpolation = cv2.INTER_AREA)
            cv2.imwrite(ruta1, resized)
          os.remove(ruta)
        except Exception as e:
          print("Excepcion en ",ruta)
          print(str(e))
          continue

In [ ]:
def saveMel(signal, directory,sr):
    import gc
    gc.enable()
    # MK_spectrogram modified
    N_FFT = 1024         # 
    HOP_SIZE = 1024      #  
    N_MELS = 128          # Higher   
    WIN_SIZE = 1024      # 
    WINDOW_TYPE = 'hann' # 
    FEATURE = 'mel'      # 
    FMIN = 1400

    fig = plt.figure(1,frameon=False)
    fig.set_size_inches(6,6)

    ax = plt.Axes(fig, [0., 0., 1., 1.])
    ax.set_axis_off()
    fig.add_axes(ax)
    
    S = librosa.feature.melspectrogram(y=signal, sr=sr,
                                        n_fft=N_FFT,
                                        hop_length=HOP_SIZE, 
                                        n_mels=N_MELS, 
                                        htk=True, 
                                        fmin=FMIN, # higher limit ##high-pass filter freq.
                                        fmax=sr/2) # AMPLITUDE
    librosa.display.specshow(librosa.power_to_db(S**2,ref=np.max), fmin=FMIN) #power = S**2

    fig.savefig(directory)
    plt.ioff()
    #plt.show(block=False)
    fig.clf()
    ax.cla()
    plt.clf()
    plt.close('all')

In [ ]:
def PrimerMetodo(ruta):
  path, fichero = os.path.split(ruta)
  MismaDuracion = path.replace("Original","MismaDuracion")
  existe = os.path.exists(MismaDuracion)#Se optiene si existe o no
  if not existe:  
    os.makedirs(MismaDuracion)
  IgualarDuraciones(ruta,DURACION_MAXIMA)
  Espectrograma1()
  shutil.rmtree(original.replace("Original","MismaDuracion"))

In [ ]:
def SegundoMetodo(ruta):
  path, fichero = os.path.split(ruta)
  espectograma = path.replace("Original","Espectrogramas")
  existe = os.path.exists(espectograma)#Se optiene si existe o no
  if not existe:  
    os.makedirs(espectograma)
  file = os.path.splitext(fichero)[0]
  extension = os.path.splitext(fichero)[1]
  if extension == ".csv":
    return
  try:
    rutaNueva = ruta.replace("Original","Espectrogramas").replace(".wav",".png")
    s, fs = sound.load(ruta)
    dB_max = 96
    Sxx_power, tn, fn, ext = sound.spectrogram(s, fs, nperseg=1024, noverlap=1024//2)
    ext[2] = frecuenciaMinima
    ext[3] = frecuenciaMaxima
    Sxx_power , fn = filtroFrecuencias(Sxx_power, fn, frecuenciaMaxima,frecuenciaMinima)

    Sxx_power_noNoise= sound.median_equalizer(Sxx_power, display=False, **{'extent':ext})
    Sxx_db_noNoise = power2dB(Sxx_power_noNoise)

    Sxx_db_noNoise_smooth = sound.smooth(Sxx_db_noNoise, std=0.5,
                                display=False, savefig=None,
                                **{'vmin':0, 'vmax':dB_max, 'extent':ext})
    im_mask = rois.create_mask(im=Sxx_db_noNoise_smooth, mode_bin ='relative',
                                  bin_std=8, bin_per=.5,
                                  verbose=False, display=False)

    im_rois, df_rois = rois.select_rois(im_mask, min_roi=25, max_roi=None,
                                        display= False,
                                        **{'extent':ext})

    df_rois = format_features(df_rois, tn, fn)


    area = (df_rois.max_x - df_rois.min_x) * (df_rois.max_y - df_rois.min_y)
    df_rois["area"] = area
    ancho = (df_rois.max_x - df_rois.min_x)
    df_rois["ancho"] = ancho
    duracion = (df_rois.max_t - df_rois.min_t)
    df_rois["duracion"] = duracion

    #df_rois = df_rois[ (df_rois['area'] >= 100) & (df_rois['duracion'] >= 1)]
    if(len(df_rois) == 0):
      imageio.imwrite(rutaNueva, (Sxx_db_noNoise_smooth).astype(np.uint8))
      img = iguala(cv2.imread(rutaNueva, cv2.IMREAD_GRAYSCALE))
      cv2.imwrite(rutaNueva, img)
      return
    Sxx_db = Sxx_db_noNoise_smooth
    #df_shape, params = features.shape_features(Sxx_db, resolution='low', rois=df_rois)
    #df_centroid = features.centroid_features(Sxx_db, df_rois)

    #median_freq = fn[np.round(df_centroid.centroid_y).astype(int)]
    #df_centroid['centroid_freq'] = median_freq/fn[-1]

    #from sklearn.manifold import TSNE

    #X = df_shape.loc[:,df_shape.columns.str.startswith('shp')]
    #X = X.join(df_centroid.centroid_freq) # add column and normalize values

    #tsne = TSNE(n_components=2, perplexity=12, init='pca', verbose=True)
    #Y = tsne.fit_transform(X)

    #from sklearn.cluster import DBSCAN
    #cluster = DBSCAN(eps=5, min_samples=4).fit(Y)

    #df_rois['label'] = cluster.labels_.astype(str)

    #from collections import Counter
    #recuento = Counter(df_rois['label'])
    #recuento=recuento.most_common(1)
            
    #df_rois = df_rois[ (df_rois['label'] == recuento[0][0])]
    df_rois = agrupaRois(df_rois)
    area = (df_rois.max_x - df_rois.min_x) * (df_rois.max_y - df_rois.min_y)
    df_rois["area"] = area
    ancho = (df_rois.max_x - df_rois.min_x)
    df_rois["ancho"] = ancho
    duracion = (df_rois.max_t - df_rois.min_t)
    df_rois["duracion"] = duracion
    #Metodo primero descrito
    # df_rois.sort_values(by = "area", ascending=False, inplace=True)
    # indice = 0
    # imagenGuardar = Sxx_db_noNoise_smooth
    # for index, row in df_rois.iterrows():
    #     if indice >= ROIS_AUDIO:
    #         break
    #     rutaNueva = os.path.join(espectograma,file+str(indice)+".png")
    #     imagenGuardar = Sxx_db_noNoise_smooth[:,row["min_x"]:row["max_x"]]
    #     imageio.imwrite(rutaNueva, (imagenGuardar).astype(np.uint8))
    #     indice = indice + 1
    #     img = iguala(cv2.imread(rutaNueva, cv2.IMREAD_GRAYSCALE))
    #     cv2.imwrite(rutaNueva, img)
    
    df_rois.sort_values(by = "min_x", ascending=True, inplace=True)
    indice = 0
    imagenGuardar = Sxx_db_noNoise_smooth
    for index, row in df_rois.iterrows():
      if indice == 0:
        imagenGuardar = Sxx_db_noNoise_smooth[0:192,row["min_x"]:row["max_x"]]
      else:
        imagenGuardar = np.concatenate((imagenGuardar,Sxx_db_noNoise_smooth[0:192,row["min_x"]:row["max_x"]]), axis=1)
      indice = indice + 1
    imageio.imwrite(rutaNueva, (imagenGuardar).astype(np.uint8))
    img = iguala(cv2.imread(rutaNueva, cv2.IMREAD_GRAYSCALE))
    cv2.imwrite(rutaNueva, img)
  except Exception as e:
    print('Failed: '+ str(e))

In [ ]:
def TercerMetodo(ruta):
  path, fichero = os.path.split(ruta)
  MismaDuracion = path.replace("Original","MismaDuracion")
  existe = os.path.exists(MismaDuracion)#Se optiene si existe o no
  if not existe:  
    os.makedirs(MismaDuracion)
  IgualarDuraciones(ruta,DURACION_MAXIMA)
  Espectrograma2()
  shutil.rmtree(original.replace("Original","MismaDuracion"))

In [ ]:
def CuartoMetodo(ruta):
  path, fichero = os.path.split(ruta)
  MismaDuracion = path.replace("Original","MismaDuracion")
  existe = os.path.exists(MismaDuracion)#Se optiene si existe o no
  if not existe:  
    os.makedirs(MismaDuracion)
  IgualarDuraciones(ruta,DURACION_MAXIMA)
  EspectrogramaMel()
  shutil.rmtree(original.replace("Original","MismaDuracion"))

In [ ]:
frecuenciaMaxima = 10000
frecuenciaMinima = 1000
DURACION_MAXIMA = 10 #EN SEGUNDOS
metodo1=PrimerMetodo
metodo2=SegundoMetodo
metodo3=TercerMetodo
metodo4=CuartoMetodo
procesa = metodo2
ROIS_AUDIO = 50
SOBREESCRIBIR = False
ANCHO = 265
ALTO = 192
contador = 0
path = os.path.join('drive',"MyDrive","Colab Notebooks", "Cuaderno1","DataSet")
original=os.path.join(path,"Original")
entrenamiento=os.path.join(original,"train")
test=os.path.join(original,"test")
val=os.path.join(original,"val")
existe = os.path.exists(entrenamiento)
if not existe:  
  os.makedirs(entrenamiento) 
existe = os.path.exists(test)
if not existe:  
  os.makedirs(test)
existe = os.path.exists(val)
if not existe:  
  os.makedirs(val)

In [ ]:
fTotales = ficherosTotales()

In [ ]:
for nombre_directorio, dirs, ficheros in os.walk(original):
    for nombre_fichero in ficheros:
        ruta=os.path.join(nombre_directorio,nombre_fichero)
        procesa(ruta)
        contador = contador + 1
        sys.stdout.write('\r')
        sys.stdout.write('%2.0f%% ' %np.round(((contador+1)/fTotales*100)))
        sys.stdout.write('('+str(contador)+' de '+str(fTotales)+')')

100% (899 de 899)

In [ ]:
for nombre_directorio, dirs, ficheros in os.walk(original.replace("Original","Espectrogramas")):
  fich = []
  for nombre_fichero in ficheros:
    fich.append(float(nombre_fichero.replace(".png","")))
  fich.sort()
  for i in fich[:-300]:
    pass#os.remove(os.path.join(nombre_directorio,str(i)+".png"))

ValueError: ignored